# $100 S&P500 investment test

Suppose you are an investor with $100 in 2005. Write a python program that calculates the total return (including dividends) if you invested this money into the S&P500. If you could erase any continuous 11 calendar day period from this history in order to get the best returns, what would your new total return be?

In [40]:
import yfinance as yf

def remove_worst_n_day_period(df, sv=100, n=11):
    """
    Remove the worst consecutive n-day period from a price history and recalculate portfolio value.

    Parameters:
        df (DataFrame): Price history with 'Close' column.
        sv (float): Starting value of investment (default $100).
        n (int): Number of consecutive days to remove (default 11).

    Returns:
        df_filtered (DataFrame): DataFrame after removing worst period with portfolio value.
        original_total (float): Total portfolio value without removing any days.
        new_total (float): Total portfolio value after removing worst period.
    """
    df = df.copy()
    df["Daily Return"] = df["Close"].pct_change()
    df["Growth Factor"] = 1 + df["Daily Return"]
    
    # Rolling n-day cumulative return
    df["Rolling Return"] = df["Growth Factor"].rolling(window=n).apply(lambda x: x.prod() - 1, raw=True)
    
    # Identify worst n-day period
    worst_end_idx = df["Rolling Return"].idxmin()
    pos = df.index.get_loc(worst_end_idx)
    worst_period_idx = df.iloc[pos - n + 1 : pos + 1].index
    
    # Drop worst period
    df_filtered = df.drop(worst_period_idx)
    
    # Recalculate cumulative portfolio value
    df_filtered["Portfolio Value"] = sv * (1 + df_filtered["Daily Return"]).cumprod()
    original_total = sv * (1 + df["Daily Return"]).cumprod().iloc[- 1]
    new_total = df_filtered["Portfolio Value"].iloc[- 1]
    
    return df_filtered, original_total, new_total

# Example usage
ticker = yf.Ticker("^SPX")
hd = ticker.history(start="2005-01-01", auto_adjust=True)

hd_filtered, original_total, new_total = remove_worst_n_day_period(hd, sv=100, n=11)
print(f"Original total return: ${original_total:.2f}")
print(f"New total return after erasing worst period: ${new_total:.2f}")


Original total return: $529.93
New total return after erasing worst period: $712.59
